In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from pythainlp import sent_tokenize, word_tokenize

In [4]:
docs = pd.read_csv('datasets/Open source [WeVis-Promise Tracker] Data - promise.csv')

In [5]:
docs

,promiseId,party,topic,promiseTitle,status,explain,isNCPO,refPicture,picturesDrive,vdo,nameLink1,urlLink1,nameLink2,urlLink2,nameLink3,urlLink3,nameLink4,urlLink4
0,1,พลังไทยรักไทย,economics,1 จังหวัด 1 แหล่งท่องเที่ยว,nodata,ซึ่งไม่พบข้อมูล 1 จังหวัด 1 แหล่งท่องเที่ยว แต...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,ไฮไลต์ 77 จังหวัดและสถานที่ท่องเที่ยว - ไทยรัฐ,https://www.thairath.co.th/lifestyle/travel/th...,NaN,NaN,NaN,NaN
1,2,พลังไทยรักไทย,equality,1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ,nodata,ซึ่งไม่พบข้อมูล1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,"อว. เดินหน้าโครงการ ""อว. ลดค่าเทอม"" - กระทรวงก...",https://www.mhesi.go.th/index.php/news-and-ann...,NaN,NaN,NaN,NaN
2,3,พลังไทยรักไทย,economics,1 ตำบล 1 ธนาคาร,nodata,ซึ่งไม่พบข้อมูล 1 ตำบล 1 ธนาคาร,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,พลังไทยรักไทย,equality,30 บาทรักษาทุกโรคได้จริง,nodata,ซึ่งไม่พบข้อมูลเกี่ยวกับ 30 บาทรักษาทุกโรคได้จ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,จุดเริ่มต้นของโครงการ 30 บาทรักษาทุกโรค,https://www.nhso.go.th/page/history,สปสช. เห็นชอบให้มีการ “ยกระดับระบบหลักประกันสุ...,https://www.nhso.go.th/news/3368,นายศุภชัย ใจสมุทร ส.ส.บัญชีรายชื่อและนายทะเบีย...,https://www.facebook.com/suphachai.jaismut/pos...
4,5,พลังปวงชนไทย,equality,30 รักษาทั่วไทย,nodata,30 บาทรักษาทุกโรคเป็นนโยบายของพรรคไทยรักไทยตั้...,False,พลังปวงชนไทย_1,https://drive.google.com/drive/folders/1snFv2j...,-,จุดเริ่มต้นของโครงการ 30 บาทรักษาทุกโรค,https://www.nhso.go.th/page/history,สปสช. เห็นชอบให้มีการ “ยกระดับระบบหลักประกันสุ...,https://www.nhso.go.th/news/3368,นายศุภชัย ใจสมุทร ส.ส.บัญชีรายชื่อและนายทะเบีย...,https://www.facebook.com/suphachai.jaismut/pos...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,351,ชาติไทยพัฒนา,equality,ให้ลูกหลานเกษตรกรเรียนฟรีถึงปริญญาตรี (หากทำงา...,working,กรมส่งเสริมสหกรณ์ได้จัดทำโครงการนำลูกหลานเกษตร...,False,ชาติไทยพัฒนา_4,https://drive.google.com/drive/folders/1snFv2j...,-,โครงการนำลูกหลานเกษตรกรกลับบ้าน สานต่ออาชีพการ...,https://drive.google.com/file/d/1BLEXX13lot6_D...,รายละเอียดโครงการนำลูกหลานเกษตรกรฯ - กรมส่งเสร...,https://web.cpd.go.th/nakhonphanom/images/PDF_...,NaN,NaN,NaN,NaN
325,352,ชาติพัฒนา,equality,ให้สิทธิ์ในการนำค่ารักษาพยาบาลของผู้สูงอายุหรื...,nodata,ให้สิทธิ์ในการนำค่ารักษาพยาบาลของผู้สูงอายุหรื...,False,ชาติพัฒนา_2,https://drive.google.com/drive/folders/1snFv2j...,-,ชาติพัฒนาเป็นทางเลือก ในการก้าวข้ามความขัดแย้ง...,https://www.bangkokbiznews.com/advertorials/ne...,ชาติพัฒนาย้ำเสถียรภาพรัฐบาล-สภา แก้ปัญหาการเมื...,https://www.sanook.com/news/8496570/,ผู้มีเงินได้มีสิทธิหักลดหย่อนอะไรได้บ้าง?,https://www.rd.go.th/557.html,NaN,NaN
326,353,พลังประชารัฐ,equality,ออกเอกสารสิทธิในที่ดินกรรมสิทธิ์ พิสูจน์สิทธิ์...,working,นโยบายนี้ต้องการแก้ปัญหาการถือครองที่ดินของนัก...,False,พลังประชารัฐ_9,https://drive.google.com/drive/folders/1snFv2j...,-,‘คทช.’ แต่งตั้งกรรมการพิสูจน์สิทธิที่ดิน - มติชน,https://www.matichon.co.th/politics/news_2647539,การประชุมคณะกรรมการนโยบายที่ดินแห่งชาติ (คทช.)...,https://www.onep.go.th/%E0%B8%81%E0%B8%B2%E0%B...,สรุปผลการประชุมคณะรัฐมนตรี วันที่ 21 ธันวาคม ...,https://www.ryt9.com/s/cabt/3283622,NaN,NaN
327,354,พลังไทยรักไทย,environmental,อีสานเขียวภาค 2 เมกะโปรเจค,nodata,ซึ่งไม่พบความเคลื่อนไหวเกี่ยวกับอีสานเขียวภาค ...,False,พลังไทยรักไทย_2,https://drive.google.com/drive/folders/1snFv2j...,-,นโยบายพรรพลังไทยรักไทย - line today,https://today.line.me/th/v2/article/YWmgPL?ima...,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
doc_matrix = vectorizer.fit_transform(docs['promiseTitle'].values)

In [7]:
doc_matrix[0]

<1x1062 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [8]:
cosine_similarity(doc_matrix[0],doc_matrix[3])

array([[0.03962221]])

In [9]:
group_word = []
for i,base in enumerate(doc_matrix):
    mk = []
    for j,target in enumerate(doc_matrix):
        if cosine_similarity(base,target)[0][0] >= 0.35:
            if docs['promiseTitle'].values[j] == docs['promiseTitle'].values[i]:
                mk.append(docs['promiseTitle'].values[i])
                continue
            else :
                mk.append(docs['promiseTitle'].values[j])
                mk.append(docs['promiseTitle'].values[i])
    if len(mk) != 0:
        group_word.append(mk)

In [10]:
group_word[:10]

[['1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ธนาคาร',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'นโยบาย 1 ตำบล 1 นักศึกษาแพทย์',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'เพิ่มนักโภชนาการ 1 คน ต่อ 1 เขต การศึกษา',
  '1 จังหวัด 1 แหล่งท่องเที่ยว',
  'สร้าง co-working space ใน กทม. 1 แขวง 1 ที่ ไม่คิดค่าใช้จ่าย ไวไฟฟรี',
  '1 จังหวัด 1 แหล่งท่องเที่ยว'],
 ['1 จังหวัด 1 แหล่งท่องเที่ยว',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  '1 ตำบล 1 ธนาคาร',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'นโยบาย 1 ตำบล 1 นักศึกษาแพทย์',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'เพิ่มนักโภชนาการ 1 คน ต่อ 1 เขต การศึกษา',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'รักษาฟรีทุกโรค แบบมีคุณภาพ พร้อมรถตู้พยาบาลประจำตำบล ตำบลละ 1 คัน',
  '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
  'สร้าง co-working space ใน กทม. 1 แขวง 1 ที่ ไม่คิดค่

In [13]:
def check_similar(list1,list2):
    count = 0
    for i in list1:
        if i in list2:
            count += 1
    if (count/len(list2))*100 >= 80:
        return True
    else:
        return False

In [14]:
check_similar(group_word[1],group_word[0])

True

# Kmeans

In [15]:
kmeans = KMeans(n_clusters=24)
clustered_docs = kmeans.fit_predict(doc_matrix)

In [16]:
clustered_docs

array([22, 22, 22, 23,  7,  4,  1,  2,  3,  1,  2,  5, 23,  6,  9,  0,  5,
        2, 17, 13, 20, 16, 21,  0, 20, 20, 20, 17, 20, 11, 20, 20, 12, 12,
       17, 12, 12, 12, 16, 19, 17, 19, 19,  8, 19,  6, 15,  1, 10, 23, 17,
        1,  5,  1,  1, 17, 22,  9, 13, 17,  5,  5,  5,  5,  5,  4,  7, 21,
       21,  5, 20,  2, 23,  6,  2,  0, 22,  0, 11, 17,  3, 18, 17, 11, 22,
       17, 10, 15, 21,  8,  9, 23,  9, 17,  9, 19, 18, 22,  2, 22, 18, 10,
        4,  4,  3,  1, 21, 23, 22, 22, 15, 22,  0,  9, 17, 11, 11, 16,  1,
       17, 13, 14, 17, 20,  2, 17, 19,  1,  1,  1, 17, 17, 13, 13,  8,  8,
        5, 14, 14, 14, 22,  2, 16, 17,  0, 16, 16,  1, 17, 16, 12,  1, 17,
        9,  9,  7, 11, 18,  9, 20, 19,  2, 19, 11, 17,  5, 17, 11,  4, 17,
       23, 17,  4,  6,  8, 12, 16, 16, 16, 17, 17, 23, 17,  5, 20,  1,  1,
       17,  8,  5, 22,  1,  1,  1,  1,  1,  7,  6, 17,  4, 17, 10,  6, 14,
       10,  1, 23,  8,  8,  3, 21,  2,  4, 20,  3,  8, 22,  3, 11, 17,  8,
        8,  8, 20, 13, 20

In [17]:
newdf = pd.DataFrame(columns=['data','party','label_clusters'],data=np.array([docs['promiseTitle'].values,docs['party'].values,clustered_docs]).T)

In [ ]:
all_df = newdf[newdf['label_clusters'] == 0]
for i in range(1,24):
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])

In [19]:
all_df

,data,party,label_clusters
15,การค้าออนไลน์อาชีพสงวนคนไทย,เศรษฐกิจใหม่,0
23,เกิดปั๊บรับแสน,ประชาธิปัตย์,0
75,โฉนดใบเดียวทั้งแผ่นดิน,เพื่อชาติ,0
77,ชนเผ่าเข้าสภา,ประชาภิวัฒน์,0
112,นโยบายทลายกำแพงใจ,เพื่อชาติ,0
...,...,...,...
107,"ทุนการศึกษาครูเทคโนโลยีทุกอำเภอ อำเภอละ 1,000,...",ชาติพัฒนา,23
170,"ผลักดันใช้ยางในประเทศ 20% ปีละ 1,000,000 ตัน",ประชาธิปัตย์,23
181,พัฒนาขีดความสามารถในการรักษาพยาบาลของโรงพยาบาล...,รวมพลังประชาชาติไทย,23
206,เพิ่มโอกาสทำมาหากินของทุกคนทุกพื้นที่ด้วยเทคโน...,พลเมืองไทย,23


In [ ]:
all_df = newdf[newdf['label_clusters'] == 0]
for i in range(1,24):
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])
mk_list = []
for i in range(1,24):
    subdict = dict()
    subdict['group'] = str(i)
    party = all_df[all_df['label_clusters'] == i][['party']].values
    title = all_df[all_df['label_clusters'] == i][['data']].values
    policy = []
    for i in range(len(party)):
        policy_dict = dict()
        policy_dict['party'] = party[i][0]
        policy_dict['title'] = title[i][0]
        policy.append(policy_dict)
    subdict['policy'] = policy
    mk_list.append(subdict)

In [21]:
mk_list

[{'group': '1',
  'policy': [{'party': 'ประชาภิวัฒน์',
    'title': 'Single mom 10,000 บาทต่อเดือน'},
   {'party': 'ก้าวไกล/อนาคตใหม่', 'title': 'กระจายรายได้ส่วนกลางต่อท้องถิ่น'},
   {'party': 'ประชาธิปัตย์', 'title': 'ค่าตอบแทน อสม. เดือนละ 1,200 บาท'},
   {'party': 'ประชาธิปัตย์', 'title': 'คูปองเพิ่มทักษะสำหรับผู้ใหญ่'},
   {'party': 'พลังประชารัฐ', 'title': 'งานผู้สูงอายุ 1,000,000 ตำแหน่ง'},
   {'party': 'เพื่อชาติ', 'title': 'เงินช่วยเหลือผู้สูงวัย 2,000 บาทต่อเดือน'},
   {'party': 'เพื่อไทย',
    'title': 'ทำให้ราคาสินค้าเกษตร เพิ่ม 30% ภายใน 6 เดือน'},
   {'party': 'พลังประชารัฐ',
    'title': 'นำรถเก่าแลกรถไฟฟ้าได้ส่วนลด 100,000 บาท'},
   {'party': 'ประชาธิปัตย์',
    'title': 'เบี้ยผู้ยากไร้ 800 บาท โอนตรงเข้าบัญชี'},
   {'party': 'ประชาภิวัฒน์',
    'title': 'เบี้ยผู้สูงอายุ (คนชรา) 5,000 บาทต่อเดือน'},
   {'party': 'ประชาธิปัตย์', 'title': 'เบี้ยยังชีพผู้สูงอายุ 1,000 บาท'},
   {'party': 'ชาติพัฒนา',
    'title': 'ปรับฐานเงินเดือนกำนัน ผู้ใหญ่บ้าน ผู้ช่วยผู้ใหญ่บ้าน สารวัต

In [20]:
all_df.to_csv('mk.csv',index=False)